In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import zipfile
import datetime
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'data_info_val_sample_submission.csv', 'categories.json']


In [2]:
!pip install pytorch_pretrained_bert 

    100% |████████████████████████████████| 122kB 4.8MB/s ta 0:00:01


In [3]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification

In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

100%|██████████| 995526/995526 [00:00<00:00, 1832875.07B/s]


In [6]:
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
tokenized_text

['[CLS]',
 'who',
 'was',
 'jim',
 'hen',
 '##son',
 '?',
 '[SEP]',
 'jim',
 'hen',
 '##son',
 'was',
 'a',
 'pu',
 '##ppet',
 '##eer',
 '[SEP]']

In [7]:

TRAIN_CSV_PATH = '../input/train.csv'
TEST_CSV_PATH = '../input/test.csv'

    
train = pd.read_csv(TRAIN_CSV_PATH, index_col='itemid')
test = pd.read_csv(TEST_CSV_PATH, index_col='itemid')

train.rename(columns={'Category': 'label'},inplace = True)
test.rename(columns={'Category': 'label'},inplace = True)
cols = ['title', 
        'label']
train = train.loc[:, cols]
test = test.loc[:, cols]
train.fillna('class_0', inplace=True)
test.fillna('class_0', inplace=True)
train.head(3)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,title,label
itemid,,
307504,nyx sex bomb pallete natural palette,0
461203,etude house precious mineral any cushion pearl...,1
3592295,milani rose powder blush,2


In [8]:
test.head()

,title,label
itemid,,
370855998,flormar 7 white cream bb spf 30 40ml,class_0
637234604,maybelline clear smooth all in one bb cream sp...,class_0
690282890,murah innisfree eco natural green tea bb cream...,class_0
930913462,loreal white perfect day cream spf 17 pa white...,class_0
1039280071,hada labo cc cream ultimate anti aging spf 35 ...,class_0


In [9]:
train['label'] =train['label'].apply(lambda x : "class_" + str(x) )
# test['title'] =test['title'].apply(lambda x : "class_" + str(x) )
# train['title'] = train['title'].astype(str).str.replace('\d+', '')
# test['title'] = test['title'].astype(str).str.replace('\d+', '')

In [10]:
train.head()

,title,label
itemid,,
307504,nyx sex bomb pallete natural palette,class_0
461203,etude house precious mineral any cushion pearl...,class_1
3592295,milani rose powder blush,class_2
4460167,etude house baby sweet sugar powder,class_3
5853995,bedak revlon color stay aqua mineral make up,class_3


In [11]:
from collections import Counter
# Counter(train.title)

In [12]:
!wget https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/run_classifier.py

--2019-03-10 16:34:16--  https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/run_classifier.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28675 (28K) [text/plain]
Saving to: ‘run_classifier.py’

run_classifier.py   100%[===================>]  28.00K  --.-KB/s    in 0.01s   

2019-03-10 16:34:16 (2.71 MB/s) - ‘run_classifier.py’ saved [28675/28675]



In [13]:
from sklearn.model_selection import train_test_split

VALIDATION_RATIO = 0.1

RANDOM_STATE = 1234

train, val= \
    train_test_split(
        train, 
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [14]:
VOCAB = '../input/bert-pretrained-models/multi_cased_L-12_H-768_A-12/vocab.txt'
MODEL = '../input/bert-pretrained-models/multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001'

In [15]:
label_list = ['class_' +str(i) for i in range(58)]
# print(label_list)

In [16]:
from run_classifier import *

train_examples = [InputExample(guid = 'train',text_a=row.title,label= row.label) for row in train.itertuples()]
val_examples = [InputExample(guid = 'val',text_a=row.title,label= row.label) for row in val.itertuples()]
test_examples = [InputExample(guid = 'test',text_a=row.title,label='class_0') for row in test.itertuples()]

len(train_examples)

599953

In [17]:
# train.head()

In [18]:
orginal_total = len(train_examples)
train_examples = train_examples[:int(orginal_total*0.2)]

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
gradient_accumulation_steps = 1
train_batch_size = 32
eval_batch_size = 128
train_batch_size = train_batch_size // gradient_accumulation_steps
output_dir = 'output'
bert_model = 'bert-base-multilingual-cased'
num_train_epochs = 3
num_train_optimization_steps = int(
            len(train_examples) / train_batch_size / gradient_accumulation_steps) * num_train_epochs
cache_dir = "model"
learning_rate = 5e-5
warmup_proportion = 0.1
max_seq_length = 64


In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
              cache_dir=cache_dir,
              num_labels = 58)
model.to(device)
if n_gpu > 1:
    model = torch.nn.DataParallel(model)
model, tokenizer

03/10/2019 16:34:20 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /tmp/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
03/10/2019 16:34:21 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz not found in cache, downloading to /tmp/tmpfmo3ol0z
100%|██████████| 662804195/662804195 [00:10<00:00, 65178381.74B/s]
03/10/2019 16:34:31 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpfmo3ol0z to cache at model/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
03/10/2019 16:34:34 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for model/731c19ddf94e294e00ec1ba9

(BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(119547, 768)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): BertLayerNorm()
       (dropout): Dropout(p=0.1)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0): BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): BertLayerNorm()
               (dropout): Dropout(p=0.1)
             )
           )
           

In [21]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=learning_rate,
                             warmup=warmup_proportion,
                             t_total=num_train_optimization_steps)

In [22]:
tokenizer

In [23]:
# label_list

In [24]:
train_features = convert_examples_to_features(
    train_examples, label_list, max_seq_length, tokenizer)

03/10/2019 16:34:53 - INFO - run_classifier -   *** Example ***
03/10/2019 16:34:53 - INFO - run_classifier -   guid: train
03/10/2019 16:34:53 - INFO - run_classifier -   tokens: [CLS] bahan thick s ##cu ##ba ama ##bilis off shoulder dress pes ##ta party [SEP]
03/10/2019 16:34:53 - INFO - run_classifier -   input_ids: 101 27961 59925 187 12352 10537 28149 48411 11898 78681 67348 59411 10213 14039 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/10/2019 16:34:53 - INFO - run_classifier -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/10/2019 16:34:53 - INFO - run_classifier -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/10/2019 16:34:53 - INFO - run_classifier -   label: class_22 (id = 22)
03/10/2019 16:34:53 - INFO - run_classifier - 

In [25]:
# train_examples

In [ ]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

train_features = convert_examples_to_features(
    train_examples, label_list, max_seq_length, tokenizer)
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

model.train()
for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    total_step = len(train_data) // train_batch_size
    ten_percent_step = total_step // 10
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        loss.backward()

        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            
        if step % ten_percent_step == 0:
            print("Fininshed: {:.2f}% ({}/{})".format(step/total_step*100, step, total_step))

03/10/2019 16:35:14 - INFO - run_classifier -   *** Example ***
03/10/2019 16:35:14 - INFO - run_classifier -   guid: train
03/10/2019 16:35:14 - INFO - run_classifier -   tokens: [CLS] bahan thick s ##cu ##ba ama ##bilis off shoulder dress pes ##ta party [SEP]
03/10/2019 16:35:14 - INFO - run_classifier -   input_ids: 101 27961 59925 187 12352 10537 28149 48411 11898 78681 67348 59411 10213 14039 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/10/2019 16:35:14 - INFO - run_classifier -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/10/2019 16:35:14 - INFO - run_classifier -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/10/2019 16:35:14 - INFO - run_classifier -   label: class_22 (id = 22)
03/10/2019 16:35:14 - INFO - run_classifier - 

In [ ]:
 if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save a trained model and the associated configuration
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
torch.save(model_to_save.state_dict(), output_model_file)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
with open(output_config_file, 'w') as f:
    f.write(model_to_save.config.to_json_string())

In [ ]:
# Load a trained model and config that you have fine-tuned
config = BertConfig(output_config_file)
model = BertForSequenceClassification(config, num_labels=len(label_list))
model.load_state_dict(torch.load(output_model_file))
model.to(device)  # important to specific device
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [ ]:
config

In [ ]:
# val
eval_examples = val_examples
eval_features = convert_examples_to_features(
    eval_examples, label_list, max_seq_length, tokenizer)
logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
logger.info("  Batch size = %d", eval_batch_size)
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += input_ids.size(0)
    nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / nb_eval_examples
loss = tr_loss/nb_tr_steps
result = {'eval_loss': eval_loss,
          'eval_accuracy': eval_accuracy,
          'global_step': global_step,
          'loss': loss}

output_eval_file = os.path.join(output_dir, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in sorted(result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

In [ ]:
device

In [ ]:
!ls output
! cat output/eval_results.txt

In [ ]:
model

In [ ]:
def predict(model, tokenizer, examples, label_list, eval_batch_size=128):
    model.to(device)
    eval_examples = examples
    eval_features = convert_examples_to_features(
        eval_examples, label_list, max_seq_length, tokenizer)
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_examples))
    logger.info("  Batch size = %d", eval_batch_size)
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    res = []
    for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
#         label_ids = label_ids.to(device)

        with torch.no_grad():
#             tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
#         print(logits)
        res.extend(logits.argmax(-1))
#         label_ids = label_ids.to('cpu').numpy()
#         tmp_eval_accuracy = accuracy(logits, label_ids)

#         eval_loss += tmp_eval_loss.mean().item()
#         eval_accuracy += tmp_eval_accuracy

#         nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

#     eval_loss = eval_loss / nb_eval_steps
#     eval_accuracy = eval_accuracy / nb_eval_examples
#     loss = tr_loss/nb_tr_steps 
#     result = {'eval_loss': eval_loss,
#               'eval_accuracy': eval_accuracy,
#               'global_step': global_step,
#               'loss': loss}

#     output_eval_file = os.path.join(output_dir, "eval_results.txt")
#     with open(output_eval_file, "w") as writer:
#         logger.info("***** Eval results *****")
#         for key in sorted(result.keys()):
#             logger.info("  %s = %s", key, str(result[key]))
#             writer.write("%s = %s\n" % (key, str(result[key])))
    return res
    

In [ ]:
res = predict(model, tokenizer, test_examples, label_list)

In [ ]:
set(res)

In [ ]:
predict(model, tokenizer, test_examples[:10], label_list)

In [ ]:
cat_map = {idx:lab for idx, lab in enumerate(label_list)}
res = [cat_map[c] for c  in res]

In [ ]:
cat_map

In [ ]:
res

In [ ]:
print(len(res))

In [ ]:
test.head()

In [ ]:
submission = pd.read_csv('../input/ndsc-beginner/data_info_val_sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
submission['Category'] = res
submission.head()

In [ ]:
submission['Category_1'],submission['Category_2'] = submission['Category'].str.split('_', 1).str

In [ ]:
del submission['Category']
del submission['Category_1']

In [ ]:
submission.columns = ['itemid','Category']

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv',index= False)